In [1]:
# 분별 날짜 파일에 있는 날짜 기준으로 DB에서 데이터 받아와서 저장하기

import MySQLdb
import os
import numpy as np
import csv
import datetime

# MySQL DB 연결
db = MySQLdb.connect('210.102.142.13',"root", "witlab8*", "cas_db")
c = db.cursor()

# 분별 날짜
date = np.genfromtxt('../data/date_winter.csv', delimiter=',', dtype='str')
# date = np.genfromtxt('../data/test_date_spring.csv', delimiter=',', dtype='str')

# csv 파일로 내보내기
w = open('../data/db_connect_data_am.csv', 'w', encoding='utf-8')
# w = open('../data/db_connect_test_data_am.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

# csv 파일로 내보내기
w2 = open('../data/db_connect_data_pm.csv', 'w', encoding='utf-8')
# w2 = open('../data/db_connect_test_data_pm.csv', 'w', encoding='utf-8')
wr2 = csv.writer(w2)

# 각 날짜별 데이터들 DB에서 가져오고 csv 파일로 저장
# 데이터는 cct, swr, uvb, uvi 순
for j in range(len(date)):
    sql = "select time(date), date(date), cct, cas_swr from natural_tracker left outer join cas_wave_ratio using(date) where date(date) = '"+ str(date[j][0]) + "' order by time(date)"
    c.execute(sql)
    rows = c.fetchall()
    
    # 일출 후 6시간과 일몰 후 6시간 데이터 사용, 일출 혹은 일몰 당시 데이터가 없을 경우 가장 가까운 다른 데이터로 변환
    sunrise = datetime.datetime.strptime(date[j][1], '%H:%M:%S')
    sunset = datetime.datetime.strptime(date[j][2], '%H:%M:%S')
    
    standard = datetime.datetime.strptime('1900-01-01 00:00:00', '%Y-%m-%d %H:%M:%S')
    
    sunrise = sunrise - standard
    sunset = sunset - standard
    
    start = 0
    end = 0

    for i in range(len(rows)):
        if(rows[i][0] >= sunrise):
            start = i
            print(date[j][0], rows[start][0], sunrise)
            break;
    
    for i in range(len(rows)):
        if(rows[i][0] >= sunset):
            end = i
            print(date[j][0], rows[end][0], sunset)
            break;
    
#     # start에 저장된 index부터 772개 데이터 가져와서 저장
    for l in range(start + 180, start + 360):
        wr.writerow([rows[l][2], rows[l][3]])
    print(date[j][0] + " am saved.")
    
    for l in range(end - 360, end - 180):
        wr2.writerow([rows[l][2], rows[l][3]])
    print(date[j][0] + " pm saved.")
    
    print('----------------------------------------------')
        
w.close()
w2.close()
db.close()

2017-12-13 7:35:00 7:35:00
2017-12-13 17:35:00 17:35:00
2017-12-13 am saved.
2017-12-13 pm saved.
----------------------------------------------
2017-12-27 7:42:00 7:42:00
2017-12-27 17:42:00 17:42:00
2017-12-27 am saved.
2017-12-27 pm saved.
----------------------------------------------
2017-12-28 7:43:00 7:43:00
2017-12-28 17:43:00 17:43:00
2017-12-28 am saved.
2017-12-28 pm saved.
----------------------------------------------
2018-01-03 7:44:00 7:44:00
2018-01-03 17:44:00 17:44:00
2018-01-03 am saved.
2018-01-03 pm saved.
----------------------------------------------
2018-01-05 7:45:00 7:44:00
2018-01-05 17:44:00 17:44:00
2018-01-05 am saved.
2018-01-05 pm saved.
----------------------------------------------
2018-01-06 7:44:00 7:44:00
2018-01-06 17:44:00 17:44:00
2018-01-06 am saved.
2018-01-06 pm saved.
----------------------------------------------
2018-01-07 7:44:00 7:44:00
2018-01-07 17:44:00 17:44:00
2018-01-07 am saved.
2018-01-07 pm saved.
--------------------------------

In [2]:
date = np.genfromtxt('../data/date_winter.csv', delimiter=',', dtype='str')
# date = np.genfromtxt('../data/test_date_spring.csv', delimiter=',', dtype='str')

# 데이터 받아오고 편차 계산히기
import_data = np.loadtxt('../data/db_connect_data_am.csv', delimiter=',')
# import_data = np.loadtxt('../data/db_connect_test_data_am.csv', delimiter=',')

one = import_data[:,0] # cct
two = import_data[:,1] # cas_swr

# # 날짜별 772개의 데이터 모두 저장, 2차원 배열
cct_am = []
swr_am = []

# 날짜별 데이터의 편차 (데이터 수 - 1)개의 데이터 모두 저장, 2차원 배열
delta_cct_am = []
delta_swr_am = []

# 각 날짜별 raw data 총 합 저장, 2차원 배열
cct_hap_am = []
swr_hap_am = []

# raw data들 저장
for cnt in range(len(date)): # 전체 데이터 날짜 수 (행)
    temp = []
    temp2 = []
    for i in range(180): # 한 날짜의 데이터들 한 행에 저장 (열)
        temp.append(one[i+180*cnt])
        temp2.append(two[i+180*cnt])
    cct_am.append(temp)
    swr_am.append(temp2)

# 편차 데이터 저장
for cnt in range(len(date)): 
    temp = []
    temp2 = []
    for i in range(179):
        temp.append(cct_am[cnt][i+1] - cct_am[cnt][i])
        temp2.append(swr_am[cnt][i+1] - swr_am[cnt][i])
    delta_cct_am.append(temp)
    delta_swr_am.append(temp2)

# raw data 합 저장
for i in range(len(date)):
    temp1 = 0
    temp2 = 0
    for j in range(len(cct_am[0])):
        temp1 += cct_am[i][j]
        temp2 += swr_am[i][j]
    cct_hap_am.append(temp1)
    swr_hap_am.append(temp2)

In [3]:
# 데이터 받아오고 편차 계산히기
import_data = np.loadtxt('../data/db_connect_data_pm.csv', delimiter=',')
# import_data = np.loadtxt('../data/db_connect_test_data_pm.csv', delimiter=',')

one = import_data[:,0] # cct
two = import_data[:,1] # cas_swr

# # 날짜별 772개의 데이터 모두 저장, 2차원 배열
cct_pm = []
swr_pm = []

# 날짜별 데이터의 편차 (데이터 수 - 1)개의 데이터 모두 저장, 2차원 배열
delta_cct_pm = []
delta_swr_pm = []

# 각 날짜별 raw data 총 합 저장, 2차원 배열
cct_hap_pm = []
swr_hap_pm = []

# raw data들 저장
for cnt in range(len(date)): # 전체 데이터 날짜 수 (행)
    temp = []
    temp2 = []
    for i in range(180): # 한 날짜의 데이터들 한 행에 저장 (열)
        temp.append(one[i+180*cnt])
        temp2.append(two[i+180*cnt])
    cct_pm.append(temp)
    swr_pm.append(temp2)

# 편차 데이터 저장
for cnt in range(len(date)): 
    temp = []
    temp2 = []
    for i in range(179):
        temp.append(cct_pm[cnt][i+1] - cct_pm[cnt][i])
        temp2.append(swr_pm[cnt][i+1] - swr_pm[cnt][i])
    delta_cct_pm.append(temp)
    delta_swr_pm.append(temp2)

# raw data 합 저장
for i in range(len(date)):
    temp1 = 0
    temp2 = 0
    for j in range(len(cct_pm[0])):
        temp1 += cct_pm[i][j]
        temp2 += swr_pm[i][j]
    cct_hap_pm.append(temp1)
    swr_hap_pm.append(temp2)

In [4]:
# 오전 6시 ~ 12시 데이터들의 편차 비율 계산

w = open('../data/cct_swr_calculation_am.csv', 'w', encoding='utf-8')
# w = open('../data/cct_swr_test_calculation_am.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(delta_cct_am)):
    up_dot_25 = 0
    up_dot_50 = 0
    up_dot_100 = 0
    up_dot_150 = 0
    up_dot_200 = 0
    up_dot_250 = 0
    up_dot_300 = 0
    up_dot_350 = 0
    up_dot_400 = 0
    up_dot_450 = 0
    up_dot_500 = 0

    # 위 변수들을 비율로 계산
    percent_1 = 0
    percent_2 = 0
    percent_3 = 0
    percent_4 = 0
    percent_5 = 0
    percent_6 = 0
    percent_7 = 0
    percent_8 = 0
    percent_9 = 0
    percent_10 = 0
    percent_11 = 0

    # 편차의 합, 평균, 등급 저장
    hap = 0
    avg = 0

    # 편차 절댓값을 기준으로 갯수 저장
    for j in range(len(delta_cct_am[i])):
        if(abs(delta_cct_am[i][j]) >= 25):
            up_dot_25 += 1
        if(abs(delta_cct_am[i][j]) >= 50):
            up_dot_50 += 1
        if(abs(delta_cct_am[i][j]) >= 100):
            up_dot_100 += 1
        if(abs(delta_cct_am[i][j]) >= 150):
            up_dot_150 += 1
        if(abs(delta_cct_am[i][j]) >= 200):
            up_dot_200 += 1
        if(abs(delta_cct_am[i][j]) >= 250):
            up_dot_250 += 1
        if(abs(delta_cct_am[i][j]) >= 300):
            up_dot_300 += 1
        if(abs(delta_cct_am[i][j]) >= 350):
            up_dot_350 += 1
        if(abs(delta_cct_am[i][j]) >= 400):
            up_dot_400 += 1
        if(abs(delta_cct_am[i][j]) >= 450):
            up_dot_450 += 1
        if(abs(delta_cct_am[i][j]) >= 500):
            up_dot_500 += 1
        
        # 편차 합 및 평균 저장
        hap += abs(delta_cct_am[i][j])
        avg = hap / 179

        # 편차 갯수들이 차지하는 비율 소수점 둘째짜리까지 계산
        percent_1 = round(up_dot_25 / 180 * 100, 2)
        percent_2 = round(up_dot_50 / 180 * 100, 2)
        percent_3 = round(up_dot_100 / 180 * 100, 2)
        percent_4 = round(up_dot_150 / 180 * 100, 2)
        percent_5 = round(up_dot_200 / 180 * 100, 2)
        percent_6 = round(up_dot_250 / 180 * 100, 2)
        percent_7 = round(up_dot_300 / 180 * 100, 2)
        percent_8 = round(up_dot_350 / 180 * 100, 2)
        percent_9 = round(up_dot_400 / 180 * 100, 2)
        percent_10 = round(up_dot_450 / 180 * 100, 2)
        percent_11 = round(up_dot_500 / 180 * 100, 2)
        
    # 편차의 절대값이 0.1 ~ 5이상인 경우의 갯수 저장
    up_dot_1 = 0
    up_dot_2 = 0
    up_dot_3 = 0
    up_dot_4 = 0
    up_dot_5 = 0
    up_dot_6 = 0
    up_dot_7 = 0
    up_dot_8 = 0
    up_dot_9 = 0
    up_dot_10 = 0
    up_dot_20 = 0
    up_dot_30 = 0
    up_dot_40 = 0
    up_dot_50_2 = 0

    # 위의 갯수가 하루 전체 데이터에서 차지하는 비율 저장
    percent_1 = 0
    percent_2 = 0
    percent_3 = 0
    percent_4 = 0
    percent_5 = 0
    percent_6 = 0
    percent_7 = 0
    percent_8 = 0
    percent_9 = 0
    percent_10 = 0
    percent_20 = 0
    percent_30 = 0
    percent_40 = 0
    percent_50_2 = 0

    # 편차의 합, 평균, 등급 저장
    hap_2 = 0
    avg_2 = 0

    for j in range(len(delta_swr_am[i])):
        if(abs(delta_swr_am[i][j]) >= 0.1):
            up_dot_1 += 1
        if(abs(delta_swr_am[i][j]) >= 0.2):
            up_dot_2 += 1
        if(abs(delta_swr_am[i][j]) >= 0.3):
            up_dot_3 += 1
        if(abs(delta_swr_am[i][j]) >= 0.4):
            up_dot_4 += 1
        if(abs(delta_swr_am[i][j]) >= 0.5):
            up_dot_5 += 1
        if(abs(delta_swr_am[i][j]) >= 0.6):
            up_dot_6 += 1
        if(abs(delta_swr_am[i][j]) >= 0.7):
            up_dot_7 += 1
        if(abs(delta_swr_am[i][j]) >= 0.8):
            up_dot_8 += 1
        if(abs(delta_swr_am[i][j]) >= 0.9):
            up_dot_9 += 1
        if(abs(delta_swr_am[i][j]) >= 1):
            up_dot_10 += 1
        if(abs(delta_swr_am[i][j] >= 2)):
            up_dot_20 += 1
        if(abs(delta_swr_am[i][j] >= 3)):
            up_dot_30 += 1
        if(abs(delta_swr_am[i][j] >= 4)):
            up_dot_40 += 1
        if(abs(delta_swr_am[i][j] >= 5)):
            up_dot_50_2 += 1

        hap_2 += abs(delta_swr_am[i][j])
        avg_2 = hap / 179

        percent_1 = round(up_dot_1 / 180 * 100, 2);
        percent_2 = round(up_dot_2 / 180 * 100, 2);
        percent_3 = round(up_dot_3 / 180 * 100, 2);
        percent_4 = round(up_dot_4 / 180 * 100, 2);
        percent_5 = round(up_dot_5 / 180 * 100, 2);
        percent_6 = round(up_dot_6 / 180 * 100, 2);
        percent_7 = round(up_dot_7 / 180 * 100, 2);
        percent_8 = round(up_dot_8 / 180 * 100, 2);
        percent_9 = round(up_dot_9 / 180 * 100, 2);
        percent_10 = round(up_dot_10 / 180 * 100, 2);
        percent_20 = round(up_dot_20 / 180 * 100, 2);
        percent_30 = round(up_dot_30 / 180 * 100, 2);
        percent_40 = round(up_dot_40 / 180 * 100, 2);
        percent_50 = round(up_dot_50 / 180 * 100, 2);
        
    wr.writerow([up_dot_25, up_dot_50, up_dot_100, up_dot_150, up_dot_200, up_dot_250, up_dot_300, up_dot_350, up_dot_400, up_dot_450, up_dot_500, up_dot_1, up_dot_2, up_dot_3, up_dot_4, up_dot_5, up_dot_6, up_dot_7, up_dot_8, up_dot_9, up_dot_10, up_dot_20, up_dot_30, up_dot_40, up_dot_50_2])
w.close()

In [5]:
# 색온도, 단파장의 편차 분포 저장
# 색온도 기준 적용

w = open('../data/cct_swr_calculation_pm.csv', 'w', encoding='utf-8')
# w = open('../data/cct_swr_test_calculation_pm.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range(len(delta_cct_pm)):
    up_dot_25 = 0
    up_dot_50 = 0
    up_dot_100 = 0
    up_dot_150 = 0
    up_dot_200 = 0
    up_dot_250 = 0
    up_dot_300 = 0
    up_dot_350 = 0
    up_dot_400 = 0
    up_dot_450 = 0
    up_dot_500 = 0

    # 위 변수들을 비율로 계산
    percent_1 = 0
    percent_2 = 0
    percent_3 = 0
    percent_4 = 0
    percent_5 = 0
    percent_6 = 0
    percent_7 = 0
    percent_8 = 0
    percent_9 = 0
    percent_10 = 0
    percent_11 = 0

    # 편차의 합, 평균, 등급 저장
    hap = 0
    avg = 0
    grade = 0

    # 편차 절댓값을 기준으로 갯수 저장
    for j in range(len(delta_cct_pm[i])):
        if(abs(delta_cct_pm[i][j]) >= 25):
            up_dot_25 += 1
        if(abs(delta_cct_pm[i][j]) >= 50):
            up_dot_50 += 1
        if(abs(delta_cct_pm[i][j]) >= 100):
            up_dot_100 += 1
        if(abs(delta_cct_pm[i][j]) >= 150):
            up_dot_150 += 1
        if(abs(delta_cct_pm[i][j]) >= 200):
            up_dot_200 += 1
        if(abs(delta_cct_pm[i][j]) >= 250):
            up_dot_250 += 1
        if(abs(delta_cct_pm[i][j]) >= 300):
            up_dot_300 += 1
        if(abs(delta_cct_pm[i][j]) >= 350):
            up_dot_350 += 1
        if(abs(delta_cct_pm[i][j]) >= 400):
            up_dot_400 += 1
        if(abs(delta_cct_pm[i][j]) >= 450):
            up_dot_450 += 1
        if(abs(delta_cct_pm[i][j]) >= 500):
            up_dot_500 += 1
        
        # 편차 합 및 평균 저장
        hap += abs(delta_cct_pm[i][j])
        avg = hap / 179

        # 편차 갯수들이 차지하는 비율 소수점 둘째짜리까지 계산
        percent_1 = round(up_dot_25 / 180 * 100, 2)
        percent_2 = round(up_dot_50 / 180 * 100, 2)
        percent_3 = round(up_dot_100 / 180 * 100, 2)
        percent_4 = round(up_dot_150 / 180 * 100, 2)
        percent_5 = round(up_dot_200 / 180 * 100, 2)
        percent_6 = round(up_dot_250 / 180 * 100, 2)
        percent_7 = round(up_dot_300 / 180 * 100, 2)
        percent_8 = round(up_dot_350 / 180 * 100, 2)
        percent_9 = round(up_dot_400 / 180 * 100, 2)
        percent_10 = round(up_dot_450 / 180 * 100, 2)
        percent_11 = round(up_dot_500 / 180 * 100, 2)

    # 등급 선정 기준
    if(percent_1 > 30.0):
        grade += 1
    
    if(percent_2 > 20.0):
        grade += 1
    
    if(cct_hap_pm[i] < 950000.0 or cct_hap_pm[i] > 1050000.0):
        grade += 1
        
    if(hap > 15000.0):
        grade += 1
        
        # 편차의 절대값이 0.1 ~ 5이상인 경우의 갯수 저장
    up_dot_1 = 0
    up_dot_2 = 0
    up_dot_3 = 0
    up_dot_4 = 0
    up_dot_5 = 0
    up_dot_6 = 0
    up_dot_7 = 0
    up_dot_8 = 0
    up_dot_9 = 0
    up_dot_10 = 0
    up_dot_20 = 0
    up_dot_30 = 0
    up_dot_40 = 0
    up_dot_50_2 = 0

    # 위의 갯수가 하루 전체 데이터에서 차지하는 비율 저장
    percent_1 = 0
    percent_2 = 0
    percent_3 = 0
    percent_4 = 0
    percent_5 = 0
    percent_6 = 0
    percent_7 = 0
    percent_8 = 0
    percent_9 = 0
    percent_10 = 0
    percent_20 = 0
    percent_30 = 0
    percent_40 = 0
    percent_50_2 = 0

    # 편차의 합, 평균, 등급 저장
    hap_2 = 0
    avg_2 = 0
#     grade = 0

    for j in range(len(delta_swr_pm[i])):
        if(abs(delta_swr_pm[i][j]) >= 0.1):
            up_dot_1 += 1
        if(abs(delta_swr_pm[i][j]) >= 0.2):
            up_dot_2 += 1
        if(abs(delta_swr_pm[i][j]) >= 0.3):
            up_dot_3 += 1
        if(abs(delta_swr_pm[i][j]) >= 0.4):
            up_dot_4 += 1
        if(abs(delta_swr_pm[i][j]) >= 0.5):
            up_dot_5 += 1
        if(abs(delta_swr_pm[i][j]) >= 0.6):
            up_dot_6 += 1
        if(abs(delta_swr_pm[i][j]) >= 0.7):
            up_dot_7 += 1
        if(abs(delta_swr_pm[i][j]) >= 0.8):
            up_dot_8 += 1
        if(abs(delta_swr_pm[i][j]) >= 0.9):
            up_dot_9 += 1
        if(abs(delta_swr_pm[i][j]) >= 1):
            up_dot_10 += 1
        if(abs(delta_swr_pm[i][j] >= 2)):
            up_dot_20 += 1
        if(abs(delta_swr_pm[i][j] >= 3)):
            up_dot_30 += 1
        if(abs(delta_swr_pm[i][j] >= 4)):
            up_dot_40 += 1
        if(abs(delta_swr_pm[i][j] >= 5)):
            up_dot_50_2 += 1

        hap_2 += abs(delta_swr_pm[i][j])
        avg_2 = hap / 179

        percent_1 = round(up_dot_1 / 180 * 100, 2);
        percent_2 = round(up_dot_2 / 180 * 100, 2);
        percent_3 = round(up_dot_3 / 180 * 100, 2);
        percent_4 = round(up_dot_4 / 180 * 100, 2);
        percent_5 = round(up_dot_5 / 180 * 100, 2);
        percent_6 = round(up_dot_6 / 180 * 100, 2);
        percent_7 = round(up_dot_7 / 180 * 100, 2);
        percent_8 = round(up_dot_8 / 180 * 100, 2);
        percent_9 = round(up_dot_9 / 180 * 100, 2);
        percent_10 = round(up_dot_10 / 180 * 100, 2);
        percent_20 = round(up_dot_20 / 180 * 100, 2);
        percent_30 = round(up_dot_30 / 180 * 100, 2);
        percent_40 = round(up_dot_40 / 180 * 100, 2);
        percent_50 = round(up_dot_50 / 180 * 100, 2);

        # 등급 기준 1
#     if(swr_hap[i] <= 15500.0 or swr_hap[i] >= 16500.0):
#         grade += 1

#     # 등급 기준 2
#     # 편차의 절대값이 0.1 이상인 경우가 전체 데이터의 25% + 1, 30%를 넘을 경우 +1, 40%를 넘을 경우 +2, 50%를 넘길 경우 +3
#     if(percent_1 >= 25.0):
#         grade += 1

#     if(percent_1 >= 40.0):
#         grade += 1

#     # 등급 기준 3
#     # 편차의 절대값이 1 이상인 경우가 전체 데이터의 5%를 넘을 경우 +1, 10%를 넘을 경우 +2
#     if(percent_10 >= 10.0):
#         grade += 1
        
    wr.writerow([up_dot_25, up_dot_50, up_dot_100, up_dot_150, up_dot_200, up_dot_250, up_dot_300, up_dot_350, up_dot_400, up_dot_450, up_dot_500, up_dot_1, up_dot_2, up_dot_3, up_dot_4, up_dot_5, up_dot_6, up_dot_7, up_dot_8, up_dot_9, up_dot_10, up_dot_20, up_dot_30, up_dot_40, up_dot_50_2, grade])
w.close()

In [6]:
# 색온도, 단파장 학습, 테스트 데이터 두 파일 합치기 (같은 상태의 minmaxScaler 적용을 위해)
import_data = np.genfromtxt('../data/cct_swr_calculation_am.csv', delimiter=',', dtype='float')
import_data2 = np.genfromtxt('../data/cct_swr_test_calculation_am.csv', delimiter=',', dtype='float')
w = open('../data/cct_swr_classification_am_input.csv', 'w', encoding='utf-8')
wr = csv.writer(w)

for i in range (len(import_data)):
    wr.writerow(import_data[i][:])
for i in range(len(import_data2)):
    wr.writerow(import_data2[i][:])
w.close();

In [7]:
# 텐서플로우 모델 생성 위한 import

import tensorflow as tf
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder()
tf.set_random_seed(777)

# 초기값 선정 xavier 알고리즘
def xavier_init(n_inputs, n_outputs, uniform=True):
    
    if uniform:
        init_range = tf.sqrt(6.0 / (n_inputs + n_outputs))
        return tf.random_uniform_initializer(-init_range, init_range)
    else:
        stddev = tf.sqrt(3.0 / (n_inputs + n_outputs))
        return tf.truncated_normal_initializer(stddev=stddev)

# 색온도 학습 및 테스트 (편차 분포)

import_data = np.genfromtxt('../data/cct_swr_calculation_am.csv', delimiter=',', dtype='float')

x_data = import_data[:, :25]
scaler = MinMaxScaler(feature_range=(0, 1))
x_data = scaler.fit_transform(x_data)
test_x = x_data[9:, :25]
x_data = x_data[:9, :25]

y_data = []
test_y = []

for i in range(9):
    temp = []
    temp.append(import_data[i][25])
    y_data.append(temp)

raw_y = y_data
    
for i in range(9, len(import_data)):
    temp = []
    temp.append(import_data[i][25])
    test_y.append(temp)
    

y_data = ohe.fit_transform(y_data)
y_data = y_data.toarray();

X = tf.placeholder(tf.float32, [None, 25])
Y = tf.placeholder(tf.float32, [None, 5])

nb_classes = 5

#기존 코드
# W = tf.Variable(tf.random_normal([25, nb_classes]), name='weight')
# b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# Xavier Initializer 추가 코드
W = tf.get_variable("W", shape=[25, nb_classes], initializer=xavier_init(25, nb_classes))
b = tf.Variable(tf.zeros([nb_classes]))

H = tf.nn.softmax(tf.matmul(X, W) + b)

# W1 = tf.get_variable("W1", shape=[25, 30], initializer=tf.contrib.layers.xavier_initializer())
# b1 = tf.Variable(tf.zeros([30]))
# H1 = tf.nn.relu(tf.matmul(X, W1) + b1)

# W2 = tf.get_variable("W2", shape=[30, 30], initializer=tf.contrib.layers.xavier_initializer())
# b2 = tf.Variable(tf.zeros([30]))
# H2 = tf.nn.relu(tf.matmul(H1, W2) + b2)

# W3 = tf.get_variable("W3", shape=[30, nb_classes], initializer=tf.contrib.layers.xavier_initializer())
# b3 = tf.Variable(tf.zeros([nb_classes]))
# H3 = tf.matmul(H2, W3) + b3

# H3 = tf.nn.softmax(H3)

cost = tf.reduce_mean(-tf.reduce_sum(Y * tf.log(H), axis=1))

optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    for step in range(10001):
        sess.run(optimizer, feed_dict={X:x_data, Y:y_data})
        if step % 1000 == 0:
            print(step, sess.run(cost, feed_dict={X:x_data, Y:y_data}))
            
            train_accuracy = 0
            test_accuracy = 0
            
            for i in range(len(x_data)):
                a = sess.run(H, feed_dict={X:[x_data[i]]})
                if(sess.run(tf.argmax(a, 1)) == (raw_y[i][0])):
                    train_accuracy += 1
            print("train accuracy = ", float(train_accuracy / len(x_data) * 100))

            for i in range(len(test_x)):
                a = sess.run(H, feed_dict={X:[test_x[i]]})
                if(sess.run(tf.argmax(a, 1)) == (test_y[i][0])):
                    test_accuracy += 1
                print(sess.run(tf.argmax(a, 1)), test_y[i][0])
            print("test accuracy = ", float(test_accuracy / len(test_x) * 100))

    print('-----------------------------')
    print('train_data = ', len(x_data), 'test_data = ', len(test_x))

0 1.6990889
train accuracy =  44.44444444444444
[4] 2.0
[4] 1.0
[4] 1.0
test accuracy =  0.0
1000 0.8197437
train accuracy =  88.88888888888889
[2] 2.0
[0] 1.0
[3] 1.0
test accuracy =  33.33333333333333
2000 0.54245454
train accuracy =  88.88888888888889
[2] 2.0
[0] 1.0
[3] 1.0
test accuracy =  33.33333333333333
3000 0.38935786
train accuracy =  88.88888888888889
[2] 2.0
[0] 1.0
[3] 1.0
test accuracy =  33.33333333333333
4000 0.2975467
train accuracy =  88.88888888888889
[2] 2.0
[0] 1.0
[3] 1.0
test accuracy =  33.33333333333333
5000 0.2396992
train accuracy =  88.88888888888889
[2] 2.0
[0] 1.0
[3] 1.0
test accuracy =  33.33333333333333
6000 0.20156589
train accuracy =  88.88888888888889
[2] 2.0
[0] 1.0
[3] 1.0
test accuracy =  33.33333333333333
7000 0.175273
train accuracy =  100.0
[2] 2.0
[0] 1.0
[3] 1.0
test accuracy =  33.33333333333333
8000 0.15633474
train accuracy =  100.0
[2] 2.0
[0] 1.0
[3] 1.0
test accuracy =  33.33333333333333
9000 0.14211905
train accuracy =  100.0
[2] 2.0


In [2]:
봄
[1] 일출 후 3시간 ~ 일출 후 3시간 - 6시간 정확도 78.57% 37, 14
[2] 일출 후 3시간 - 6시간 ~ 일몰 전 6시간 - 3시간 정확도 57.14% 37, 14
[3] 일몰 전 6시간 - 3시간 ~ 일몰 전 3시간 정확도 35.71% 37, 14

여름
[1] 정확도 46.67% 20, 15
[2] 정확도 53.33% 20, 15
[3] 정확도 60.00% 20, 15

가을
[1] 정확도 50%, 16, 8
[2] 정확도 62.5%, 16, 8
[3] 정확도 50%, 16, 8

겨울
[1] 정확도 0%, 9, 3
[2] 정확도 33.33%, 9, 3
[3] 정확도 66.67% 9, 3

15000